SECTION 1: THE AGENT RUNTIME (`src/agents/`, ~300 files)

This is where agents actually THINK AND ACT - the consciousness loop.


THE CORE LOOP
The agent loop is NOT a TRADITIONAL `while(true)` loop. It's EVENT-DRIVEN AND
REQUEST-RESPONSE BASED, managed by the `pi-agent-core` SDK.


ENTRY POINT: `runEmbeddedPiAgent()` in `src/agents/pi-embedded-runner/run.ts:137`

The flow for every single turn:



   User Messages Arrives
   > runEmbeddedPiAgent - L137
      > Resolve workspace & session
      > Load/validate model config
      > resolve auth profiles with fallback
      > Enter retry loop
         > runEmbeddedAttempt - L399
            > Load session file from disk
            > Create all tools
            > Build system prompt
            > Create agent session object
            > Apply system prompt override
            > Subscribe to streaming events
            > Call SDK streaming to LLM
               > Events stream back:
                  > message_start
                  > message_update
                  > message_end
                  > tool_execution_start
                  > tool_execution_update
                  > tool_execution_end
                  > agent_end
            > Accumulate response
      > Handle errors  (L 474-797)
      > Return or retry   

Response Payload(s) sent back





THE "THINKING" -- It's Delegated

The actual LLM thinking is NOT in OpenClaw code. It's delegated to the Anthopic
SDK (or equivalent provider). What OpenClaw does it:
   1. Sent message to LLM with tools and system prompt
   2. LLM processes, decides what to do, generates response
   3. SDK streams back events
   4. OpenClaw subscribes to events and handles them




The event handlers live in `pi-embedded-subscribe.handlers.ts` (L 24-61), which
dispatches to:
   - `handleMessageStart/Update/End` (in `handlers.messages.ts`)
   - `handleToolExecutionStart/Update/End` (in `handlers.tools.ts`)
   - `handleAgentStart/End` (in `handlers.lifecycle.ts`)
   

In [ ]:
+-------------------------------------+
|                 ___                 |
|    __.--/)  .-~~   ~~>>>>>>>>   .-. |
|   (._\~  \ (        ~~>>>>>>>>.~.-' |
|     -~}   \_~-,    )~~>>>>>>>' /    |
|       {     ~/    /~~~~~~. _.-~     |
|        ~.(   `--~~/      /~ ~.      |
|   .--~~~~_\  \--~(   -.-~~-.  \     |
|   ```-'~~ /  /    ~-.  \ .--~ /     |
|        (((_.'    (((__.' ```-'      |
+-------------------------------------+

---

- Pi-mono (`badlogic/pi-mono`) is an FOSS TypeScript monorepo that serves as a
  toolkit for building, managing, and deploying AI agents. It is heavily 
  assosciatied with Pi, a minimal terminal-based coding agent designed by Mario
  to be a lean, extensible alternative to more complex AI coding tools.

- In the context of PI-MOO (a TS monorepo for an AI coding agent), "streaming
  agents" refer to the core functionality of the AI, which incrementally 
  processes data, generates text, execute tools., and updates the Terminal UI
  (TUI) in real-time. The agent utilises TS, npm workspaces, and support 
  multi-LLM provider LLMs.

  1. `pi-agent-core` (The Core Engine)
     - AGENT LOOP: Handles the main loop of the AI agent, including tool 
       execution, validation, and event streaming.
     - AgentMessage: The agent operates with a flexible `AgentMessage` type
       that supports standard LLM messages (user, assistant, tool result) and
       custom app-specific types.
     - STREAMING BEHAVIOR: When the agent is streaming, it manages how messages
       are handled, requiring `streamingBehavior` (such as `steer` or `followUp`)
       if a command is sent during an active stream.

  2. `pi-tui` (STREAMING OUTPUT)
     - REAL-TIME Interaction: The Terminal UI (TUI) framework provides "almost"
       flicker-free updates, rendering streaming text, tool output, and 
       markdown in real-time.
     - Differential Rendering: The TUI updates only the necessary parts of the
       screen, providing a smooth user experience while the model streams output
                  
  


  - ... "subscribing to events" refers to an event-driven, rather than 
    poll-based, architecture where the agent actively listens for specific, 
    typed signals (WebSocket frames) from the GateWay.

    This mechanism allows the system to react instantly to changes rather than
    constantly asking "what's new?".

    Here is what "subscribe to event" specifically means for OpenClaw:
       - REAL-TIME RESPONSIVENESS: Instead of polling, the agent is notified
         immediately when specific events occur, such as incoming messages from
         connected apps (WhatsApp, Telegram, Slack), or when internal state 
         changes.
       - EVENT TYPES: Agents can subscribe to various event types handled by the
         Gateway, including `agent`, `presence`, `health` and `tick` (time-based
         heartbeats).
       - AUTOMATED TRIGGERS (Heartbeat): The "heartbeat" feature is a crucial
         subscribe event where the agent triggers a check, acts on information
         (like a new email or scheduled task), and then returns to an idle state
         , functioning as a 24/7 personal assistant.
       - STRUCTURED DATA INTERACTION: These subscribed events are typed and 
         validated against Json Schema (generated from TypeBox), ensuring that
         the agent only processes well-formed data.
       - PROACTIVE CAPABILITIES: Subscribing to events allows OpenClaw to move
         beyond simple chat, enabling proactive "push" actions, such as sending
         a daily briefing via a Telegram channel without being prompted.       
    
    Essentially, subscribing to events enables OpenClaw to operate as a 24/7
    automated agent that triggers actions based on external triggers 
    (messages/alerts) rather than waiting for user input.



- Handlers in the backend are functions or classes designed to process incoming
  requests (HTTP, events, etc.), execute buisness logic, and return a response.
  They act as intermediaries between raw ssytem events or API endpoints and the
  core application logic, often managing data retrieval, validation and security
  tasks.

  Key aspects of handlers in backend development:
  - HTTP REQUEST HANDLING: Web handlers receive a request object (URLs, headers)
    and a response object, filling the latter with JSON, HTML or status codes.
  - EVENT HANDLING: In event-driven architectures, handlers (or consumers) 
    respond to specific events, such as processing data from a queue or a 
    database change.
  - PROCESS FLOW: A handler typically follows a structured path: 
     receive request --> validate data --> interact with db/models --> return response
  - TYPES OF HANDLERS:
      - PIPE HANDLERS: Common in monitoring systems (like Sensu) to process
        incoming events via scripts.
      - WEB SERVICE HANDLERS: Used to handle SOAP or REST API requests, often
        managing headers and endpoints.
      - DEFAULT HANDLERS: Built-in server functions that handle static content
        or default file requests.
  
  In short, a handler is a specialised function that "handles" a particular
  input to ensure the application reacts correctly to external requests or 
  system events.
